<h1>Python pipeline for IGOR recordings<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparations" data-toc-modified-id="Preparations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparations</a></span></li><li><span><a href="#Exercise-1" data-toc-modified-id="Exercise-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><span style="color: limegreen">Exercise 1</span></a></span></li></ul></div>

## Preparations
Please install the following two packages in the Anaconda packages manager:

- neo ([neuralensemble.org/neo](http://neuralensemble.org/neo/), [Paper](https://biomedical-engineering-online.biomedcentral.com/articles/10.1186/s12938-017-0419-7))
- igor ([pypi.org/project/igor](https://pypi.org/project/igor/))

and download from the shared Nextcloud folder:

- Data/Igor_1

## <span style="color:limegreen">Exercise 1</span>
<blockquote>

1. Create a new script, that reads in all IGOR binary _ibw_-files into a variable called, e.g., `filenames`. Hint: You can use the method we have introduced in Chapter 2.4, _Python pipeline for a basic time series analysis_. Print out all read filenames.
2. Use the function `sorted` to sort the filenames in ascending order. Hint: Look-up the usage of `sorted` e.g. on [python.org](https://docs.python.org/3/howto/sorting.html) or [realpython.com](https://realpython.com/python-sort/). What do you notice?
</blockquote>

In [23]:
# Solution 2.1-2.2:
import numpy as np
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import os
from neo import io

file_path = "Data/igor_1/"
file_names = [file for file in os.listdir(file_path) \
              if file.endswith('.ibw')]
file_names = sorted(file_names)
print(f"file list (sorted): {file_names}")

file list (sorted): ['ad1_1.ibw', 'ad1_10.ibw', 'ad1_11.ibw', 'ad1_12.ibw', 'ad1_13.ibw', 'ad1_14.ibw', 'ad1_15.ibw', 'ad1_16.ibw', 'ad1_17.ibw', 'ad1_18.ibw', 'ad1_19.ibw', 'ad1_2.ibw', 'ad1_3.ibw', 'ad1_4.ibw', 'ad1_5.ibw', 'ad1_6.ibw', 'ad1_7.ibw', 'ad1_8.ibw', 'ad1_9.ibw']


Let's see how the IGOR files look like:

In [24]:
test_file = os.path.join(file_path, file_names[0])
test_igor_read = io.IgorIO(test_file).read_analogsignal()

Please try out the following commands in your console, one after another:

In [25]:
test_igor_read

AnalogSignal with 1 channels of length 20000; units dimensionless; datatype float64 
name: 'ad1_1'
annotations: {'note': b'BASELINE=-59.885831\rBASESUBTRACTED=0.000000\rREJECT=0.000000\rSTEP=0.000000'}
sampling rate: 25.0 1/s
time: 0.0 s to 800.0 s

In [26]:
test_igor_read.shape

(20000, 1)

In [27]:
test_igor_read.sampling_rate

array(25.) * 1/s

In [28]:
np.array(test_igor_read.sampling_rate) # 1/s

array(25.)

In [29]:
test_igor_read.times

array([0.0000e+00, 4.0000e-02, 8.0000e-02, ..., 7.9988e+02, 7.9992e+02,
       7.9996e+02]) * s

<blockquote>

3. Iterate over all `filenames` and read each IGOR file by using the following command:

    `current_igor_read = io.IgorIO(filepath).read_analogsignal()`
    
    Plot each read IGOR trace into one figure by plotting `current_igor_read.times` vs. `current_igor_read`. Save this figure to disk. Adjust an appropriate figure aspect ratio and provide useful labels and legends.    
4. In order to plot the average of all traces, we need to store each individual trace into a 2D numpy array, e.g. called `all_igor_reads`. Please recap our corresponding solution from Chapter 2.4, _Python pipeline for a basic time series analysis_ and apply it to the current problem. Then calculate the grand average over all traces within the same figure.
</blockquote>

In [ ]:
# Solution 2.3 and 2.4:
all_igor_reads = np.empty((test_igor_read.shape[0], 0))

fig = plt.figure(1, figsize=(10,4))
plt.clf()
# for i, file in enumerate(file_names):
for file in file_names:

    current_file = os.path.join(file_path, file)
    current_igor_read = io.IgorIO(current_file).read_analogsignal()

    all_igor_reads = \
      np.append(all_igor_reads, current_igor_read.as_array(), axis=1)

    """ You can calculate the time-array manually,

    current_sampling_rate=np.array(current_igor_read.sampling_rate)
    current_time_array = np.arange(current_igor_read.shape[0]) /
                            current_sampling_rate
    plt.plot(current_time_array, current_igor_read, label=file)

    or use the .times attribute of the read-in igor file:
    """
    plt.plot(current_igor_read.times, current_igor_read, label=file)

plt.plot(current_igor_read.times, all_igor_reads.mean(axis=1), lw=2.5, c="k", label="avrg.")
plt.xlabel("time [s]")
plt.legend(loc="best",fontsize=8)
plt.tight_layout()
plt.show()
plt.savefig(file_path + " overview.pdf")

<blockquote>

5. In order to kick-out traces without signifcant action potential spikings, we need to threshold the data: 
    - copy your solution from 2.3 and 2.4 into a new cell. 
    - rename the variable `all_igor_reads` into, e.g., `thresholed_igor_reads` and define a threshold `kick_out_threshold = -40`.
    - change the plotting procedure in that way, that only traces with a maximum magnitude above `kick_out_threshold` are plotted.
    - store these thresholded traces into the array `thresholed_igor_reads`.
</blockquote>

In [ ]:
# Solution 2.5


<blockquote>

6. In order to detect the prominent spikes in each trace, we will apply the `find_peaks` function from the `scipy` package: 
    - please make yourself a bit familiar with the `find_peaks` function on [scipy.org](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html).
    - create a new cell and apply the `find_peaks` function to the previously thresholded traces stored in `thresholed_igor_reads`. Define a minimum threshold for the peak detection of `spike_threshold = -10`.
</blockquote>

In [ ]:
# Solution 2.6
